In [5]:
# a mock keras model using NaiveDense and NaiveSequential
from NaiveSequential import NaiveSequential
from NaiveDense import NaiveDense
import tensorflow as tf

model = NaiveSequential([
    NaiveDense(input_size=28*28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])

assert len(model.weights) == 4

In [6]:
# a batch generator to iterate over the MNIST data in mini-batches 

import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [8]:
%pip install tensorflow.keras

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\krish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
# running one training step, updating the weights of the model after running it on one batch of data. we need:
# compute the predictions of the model for the images in the batch
# compute the loss value for these predictions, given the actual labels
# compute the gradient of the loss with regard to the model's weights
# move the weights by a small amnt in the direction opposite to the gradient.

# computation of gradient using GT

def one_training_step(mode, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

learning_rate = 1e-3

def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)

# in practice, we would be using Optimizer instance from Keras:

%pip install tensorflow

from tensorflow.keras import optimizers

optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights_2(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\krish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
# the full training loop
# an epoch of training: repeating the training step for each batch in the training data, and the full training loop is the repetition of one epoch.

def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter} : {loss: .2f}")

# test drive

from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 0
loss at batch 0 :  6.02
loss at batch 100 :  2.20
loss at batch 200 :  2.19
loss at batch 300 :  2.04
loss at batch 400 :  2.19
Epoch 1
loss at batch 0 :  1.88
loss at batch 100 :  1.85
loss at batch 200 :  1.81
loss at batch 300 :  1.67
loss at batch 400 :  1.80
Epoch 2
loss at batch 0 :  1.56
loss at batch 100 :  1.55
loss at batch 200 :  1.48
loss at batch 300 :  1.39
loss at batch 400 :  1.48
Epoch 3
loss at batch 0 :  1.30
loss at batch 100 :  1.31
loss at batch 200 :  1.22
loss at batch 300 :  1.18
loss at batch 400 :  1.26
Epoch 4
loss at batch 0 :  1.10
loss at batch 100 :  1.14
loss at batch 200 :  1.03
loss at batch 300 :  1.02
loss at batch 400 :  1.10
Epoch 5
loss at batch 0 :  0.96
loss at batch 100 :  1.00
loss at batch 200 :  0.90
loss at batch 300 :  0.91
loss at batch 400 :  0.98
Epoch 6
loss at batch 0 :  0.86
loss at batch 100 :  0.90
loss at batch 200 :  0.80
loss at batch 300 :  0.82
loss at batch 400 :  0.

In [ ]:
# evaluating the model

# evaluation of the model by taking the argmax of its predictions over the test images and comparing to the expected labels
import numpy as np

predictions = model(test_images)
predictions = predictions.numpy()

predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean(): .2f}")


# all this can be done in a few lines of Keras code, having a low level mental model helps us to take better benefits of the Keras API


accuracy:  0.82
